# 유사한 단어 찾기 게임
(임베딩 모델 학습 부분까지는 건너뛰고 그 다음부터 실행해야 합니다.)

1. 사전 학습된 모델 또는 적절한 데이터뎃을 찾는다
2. 워드 임베딩 모델을 학습시킨다
3. 단어 유사도가 0.8인 A,B를 랜덤 추출한다
4. A,B와 대응되는 C를 추출한다
5. D를 입력받는다

=> A:B = C:D 관계에 대응하는 D를 찾는 게임을 만든다.
ex) A:산, B:바다, C:나무, D:물

**<출력 예시>**

관계) [ 수긍 : 추락 = 대사관 : ?] <br>
모델이 예측한 가장 적합한 단어: 잠입 <br>
당신의 답변과 모델 예측의 유사도: 0.34 <br>
아쉽네요. 더 생각해보세요.

---

- 데이터 출처: https://github.com/kakaobrain/kor-nlu-datasets/tree/master

In [1]:
import pandas as pd

df = pd.read_csv(
    'word_game_data.tsv',
    sep='\t',
    engine='python',        # 파이썬 엔진이 불량 행에 관대
    on_bad_lines='skip',    # 문제 있는 줄은 건너뜀 ('warn'으로 바꾸면 경고만)
    dtype=str               # 타입 추정으로 인한 추가 오류 방지
)

ser1 = df['sentence1']
ser2 = df['sentence2']

combined_ser = pd.concat([ser1, ser2], axis=0).reset_index(drop=True)
combined_ser

0                개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다.
1         시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스...
2                         우리 번호 중 하나가 당신의 지시를 세밀하게 수행할 것이다.
3                              어떻게 아세요? 이 모든 것이 다시 그들의 정보다.
4         그래, 만약 네가 테니스화 몇 개를 사러 간다면, 나는 왜 그들이 100달러대에서 ...
                                ...                        
762701                                    캘리포니아는 더 잘할 수 없다.
762702                         그래서 원래의 많은 건물들이 편의점으로 대체되었다.
762703                 하우스보트의 전통은 영국 라지가 여전히 강해지는 동안 시작되었다.
762704                부고문은 아름다웠고 연예계에서의 그의 업적에 대해 현물로 쓰여졌다.
762705          남편이 요즘 너무 과로해서 이 근처에서 많은 일을 부탁할 용기가 나지 않는다.
Length: 762706, dtype: object

In [2]:
# 결측치 확인
combined_ser.isna().sum()

# 결측치 처리
combined_ser = combined_ser.dropna(how='any')
combined_ser.isna().sum()

0

데이터 전처리

In [3]:
import re
from konlpy.tag import Okt
from tqdm import tqdm

okt = Okt()

# 영문자 포함 여부 (영어가 1글자라도 섞이면 제거)
HAS_LATIN = re.compile(r'[A-Za-z]')

# 의미 희석 토큰(명사에도 많이 섞여 나오는 것들)
EXTRA_STOPS = {
    "적","라는","만큼","대로","듯","듯이","거","것","수","등","중",
    "가지","두","다음","및","그러나","하지만","또","또한","그리고",
    "있다","하다","되다"  
}

def tokenize_clean_nouns(sent):
    if not isinstance(sent, str) or not sent.strip():
        return []
    # 공백 정리
    sent = re.sub(r"\s+", " ", sent)
    # 품사 태깅 (정규화 + 원형복원)
    pos = okt.pos(sent, norm=True, stem=True)
    out = []
    for tok, tag in pos:
        # 1) 명사만 남김
        if tag != "Noun":
            continue
        # 2) 영어 포함 토큰 제거
        if HAS_LATIN.search(tok):
            continue
        # 3) 추가 불용어 제거
        if tok in EXTRA_STOPS:
            continue
        # 4) 한 글자 토큰 제거(원하면 조건 완화 가능)
        if len(tok) <= 1:
            continue
        out.append(tok)
    return out

preprocessed_data = [tokenize_clean_nouns(s) for s in tqdm(combined_ser)]


100%|██████████| 762666/762666 [42:53<00:00, 296.31it/s]  


In [4]:
preprocessed_data[2700]

['걱정']

임베딩 모델 학습

In [5]:
from gensim.models import FastText

embedding_model = FastText(
    sentences=preprocessed_data,
    vector_size=100,
    window=5,
    min_count=5,
    sg=0
)

embedding_model.wv.vectors.shape
embedding_model.wv.save_word2vec_format('word_game_w2v')

---

## 여기서부터 실행하면 됩니다

저장된 모델 로드해서 사용

In [6]:
# 임베딩 모델 로드
from gensim.models import KeyedVectors

load_model = KeyedVectors.load_word2vec_format('word_game_w2v')

게임 실행 함수

In [ ]:
import random

def game_start(model):
    
    def random_similar_pair(model, max_attempts=500):
        """
        반환: (word_a, word_b, 유사도)
        """
        words = list(model.index_to_key)
        for _ in range(max_attempts):
            w1 = random.choice(words)
            sims = model.most_similar(w1) 
            cands = [(w2, s) for w2, s in sims if s >= 0.8]
            if cands:
                w2, s = random.choice(cands)
                return w1, w2, float(s)
        raise RuntimeError(f"No pair found with similarity >= 0.8 after {max_attempts} attempts.")
    
    word_A, word_B, sim_score = random_similar_pair(load_model, max_attempts=1000) # A,B 랜덤추출
    word_C, C_sim_score = model.most_similar(negative=[word_A], topn=1)[0] # 대응되는 C 추출 (기준: 대척점)
    model_word = model.most_similar(word_C)[0][0] # 모델이 예측한 가장 적합한 단어


    # 플레이
    print(f"관계) {word_A}:{word_B} = {word_C}:?")
    try:
        word_D = input("단어를 입력하세요: ")
        similarity_score = model.similarity(word_C, word_D)
        print(f"모델이 예측한 가장 적합한 단어: {model_word}")
        print(f"당신의 답변과 모델 예측의 유사도: {similarity_score}")
        if similarity_score > 0.8:
            print("훌륭합니다.")
        else:
            print("아쉽네요 더 생각해보세요")
    except KeyError: 
        print("없는 단어입니다. 다시 시작하세요.")    

# 게임 실행!

In [ ]:
## 실행하면 게임이 시작됩니다!
game_start(load_model)

관계) 전시회:전시품 = 상대방:?
모델이 예측한 가장 적합한 단어: 상대성
당신의 답변과 모델 예측의 유사도: 0.7213615775108337
아쉽네요 더 생각해보세요
